In [1]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')

import hvplot.pandas

%opts magic unavailable (pyparsing cannot be imported)
%compositor magic unavailable (pyparsing cannot be imported)


In [2]:
df = pd.read_csv("/Users/nguyentoan/Documents/Project_panel/vgsales.csv")

In [3]:
df.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [4]:
df.describe()

,Rank,Year,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
count,16598.000000,16327.000000,16598.000000,16598.000000,16598.000000,16598.000000,16598.000000
mean,8300.605254,2006.406443,0.264667,0.146652,0.077782,0.048063,0.537441
std,4791.853933,5.828981,0.816683,0.505351,0.309291,0.188588,1.555028
min,1.000000,1980.000000,0.000000,0.000000,0.000000,0.000000,0.010000
25%,4151.250000,2003.000000,0.000000,0.000000,0.000000,0.000000,0.060000
50%,8300.500000,2007.000000,0.080000,0.020000,0.000000,0.010000,0.170000
75%,12449.750000,2010.000000,0.240000,0.110000,0.040000,0.040000,0.470000
max,16600.000000,2020.000000,41.490000,29.020000,10.220000,10.570000,82.740000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16598 entries, 0 to 16597
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Rank          16598 non-null  int64  
 1   Name          16598 non-null  object 
 2   Platform      16598 non-null  object 
 3   Year          16327 non-null  float64
 4   Genre         16598 non-null  object 
 5   Publisher     16540 non-null  object 
 6   NA_Sales      16598 non-null  float64
 7   EU_Sales      16598 non-null  float64
 8   JP_Sales      16598 non-null  float64
 9   Other_Sales   16598 non-null  float64
 10  Global_Sales  16598 non-null  float64
dtypes: float64(6), int64(1), object(4)
memory usage: 1.4+ MB


In [6]:
# Ensure there are no null values in the 'Year' column
if df['Year'].isnull().sum() == 0:
    # Convert 'Year' column from float to int
    df['Year'] = df['Year'].astype(int)
else:
    # Handle null values if any (e.g., fill with a default value or drop rows)
    df['Year'] = df['Year'].fillna(0).astype(int)

In [7]:
# Make DataFrame Pipeline Interactive
idf = df.interactive()

In [8]:
# Define Panel widgets
Start_year_slider = pn.widgets.IntSlider(name='Year slider', start=1980, end=2020, step=1,value=1980)
Start_year_slider

IntSlider(end=2020, name='Year slider', start=1980, value=1980)

In [9]:
# Define Panel widgets
End_year_slider = pn.widgets.IntSlider(name='Year slider', start=1980, end=2020, step=1,value=2020)
End_year_slider

IntSlider(end=2020, name='Year slider', start=1980, value=2020)

In [10]:
# Radio buttons for values measures
yaxis_sale = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['NA_Sales', 'EU_Sales','JP_Sales','Other_Sales','Global_Sales'],
    button_type='success'
)
yaxis_sale

RadioButtonGroup(button_type='success', name='Y axis', options=['NA_Sales', 'EU_Sales', ...], value='NA_Sales')

# Total Sales by Genre

In [11]:
Genre_pipeline = (
    idf[
        (idf.Year >= Start_year_slider) & (idf.Year <= End_year_slider)
    ]
    .groupby(['Genre'])[yaxis_sale].sum()
    .to_frame()
    .reset_index()
    .sort_values(by='Genre')  
    .reset_index(drop=True)
)

In [12]:
Genre_pipeline

In [22]:
bar_plot = Genre_pipeline.hvplot(kind='bar', 
                                 x='Genre', 
                                 y=yaxis_sale, 
                                 title='Total Sales by Genre')
bar_plot

# Sales by years and genre

In [13]:
Year_Genre_pipeline = (
    idf[
        (idf.Year >= Start_year_slider) & (idf.Year <= End_year_slider)
    ]
    .groupby(['Year','Genre'])[yaxis_sale].sum()
    .to_frame()
    .reset_index()
    .sort_values(by='Year')  
    .reset_index(drop=True)
)

In [49]:
#Year_Genre_pipeline

In [21]:
Bar_plot = Year_Genre_pipeline.hvplot(x = 'Year', by='Genre', y=yaxis_sale,line_width=2, title="Sales by years and genre")
Bar_plot

# Top 10 name by Sales

In [15]:
Name_pipeline = (
    idf[
        (idf.Year >= Start_year_slider) & (idf.Year <= End_year_slider)
    ]
    .groupby(['Name'])[yaxis_sale].sum()
    .to_frame()
    .reset_index()
    .sort_values(by=yaxis_sale,ascending=False)  
    .head(10)
    .reset_index(drop=True)
)

In [16]:
Name_pipeline

In [23]:
Name_bar_plot = Name_pipeline.hvplot(kind='barh', 
                                 x='Name', 
                                 y=yaxis_sale, 
                                 title='Top 10 name by Sales')
Name_bar_plot

# Top 10 Platform by Sales

In [24]:
Platform_pipeline = (
    idf[
        (idf.Year >= Start_year_slider) & (idf.Year <= End_year_slider)
    ]
    .groupby(['Platform'])[yaxis_sale].sum()
    .to_frame()
    .reset_index()
    .sort_values(by=yaxis_sale,ascending=False)  
    .head(10)
    .reset_index(drop=True)
)

In [18]:
Platform_pipeline

In [19]:
Platform_bar_plot = Platform_pipeline.hvplot(kind='barh', 
                                 x='Platform', 
                                 y=yaxis_sale, 
                                 title='Top 10 Flatform by Sales')
Platform_bar_plot

# Creating Dashboard

In [25]:

#Layout using Template
template = pn.template.FastListTemplate(
    title='Total sales project', 
    sidebar=[pn.pane.Markdown("# Total sales "), 
             pn.pane.Markdown("#### Total Sale by Genre, Year and Top 10 Name , Platform !."), 
             pn.pane.Markdown("## Settings"),   
             Start_year_slider,
            End_year_slider,
            yaxis_sale],
    main=[pn.Row(pn.Column(Bar_plot.panel(width=600), margin=(0,25)), 
                 bar_plot.panel(width=600)), 
          pn.Row(pn.Column(Name_bar_plot.panel(width=600)), 
                 pn.Column(Platform_bar_plot.panel(width=600)))],
    accent_base_color="#88d8b0",
    header_background="#88d8b0",
)
# template.show()
template.servable(); 

